In [ ]:
import numpy as np
from PIL import Image

THRESHOLD = 68

def compress_quadtree_rgb(img, x1, y1, x2, y2):

    if x1 >= x2 or y1 >= y2:
        return [0, 0, 0]

    block = img[x1:x2, y1:y2]

    max_vals = np.max(block, axis=(0,1))
    min_vals = np.min(block, axis=(0,1))

    if np.all(max_vals - min_vals <= THRESHOLD):
        return np.mean(block, axis=(0,1)).tolist()
    else:
        mx = (x1+x2)//2
        my = (y1+y2)//2

        return [
            compress_quadtree_rgb(img, x1, y1, mx, my),
            compress_quadtree_rgb(img, x1, my, mx, y2),
            compress_quadtree_rgb(img, mx, y1, x2, my),
            compress_quadtree_rgb(img, mx, my, x2, y2)
        ]

def decompress_quadtree_rgb(tree, img, x1, y1, x2, y2):
    if not isinstance(tree, list) or len(tree) != 4:
        img[x1:x2, y1:y2] = tree
        return

    mx = (x1+x2)//2
    my = (y1+y2)//2

    decompress_quadtree_rgb(tree[0], img, x1, y1, mx, my)
    decompress_quadtree_rgb(tree[1], img, x1, my, mx, y2)
    decompress_quadtree_rgb(tree[2], img, mx, y1, x2, my)
    decompress_quadtree_rgb(tree[3], img, mx, my, x2, y2)


if __name__ == "__main__":
    img = Image.open("images.jpeg").convert("RGB")
    arr = np.array(img)

    tree = compress_quadtree_rgb(arr, 0, 0, arr.shape[0], arr.shape[1])

    out = np.zeros_like(arr, dtype=np.float64)
    decompress_quadtree_rgb(tree, out, 0, 0, arr.shape[0], arr.shape[1])

    Image.fromarray(out.astype(np.uint8)).save("output.jpeg")
    print("RGB Compression Done! Output saved to 'output.jpeg'")

RGB Compression Done! Output saved to 'output.jpeg'
